<a href="https://colab.research.google.com/github/kristi-thinkful/thinkful-solutions/blob/master/python_III_pandas/manipulating_dataframes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# this code allows the display of output from multiple calls without requiring print() each time
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import pandas as pd
import warnings
warnings.filterwarnings("ignore")

## kickstarter
This is a dataset consisting of statistics about Kickstarter projects. Download the dataset [here](https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/ksprojects.csv). Create a DataFrame from the table named ksprojects. Write code that allows you to answer the following questions. Create a separate cell for each question.

1. How many rows of data are in the DataFrame?
2. What are the names and data types of the columns?
3. Do any of the columns contain null values?
4. Find all successful documentary projects and sort them by the amount pledged. Print the top 10 highest pledges.
5. Create a new column named average_per_backer and set its value to the total amount pledged / number of backers. What happened to the rows with 0 backers? How can this be dealt with?
7. Drop all rows with 0 backers then repeat the previous exercise.
8. Create a crosstab to get a count of records for each combination of state and category.  

####**Load data**

In [2]:
kickstarter_url = 'https://raw.githubusercontent.com/kristi-thinkful/thinkful-solutions/master/python_III_pandas/data/ksprojects.csv'
kickstarter = pd.read_csv(kickstarter_url)
kickstarter.head()

,id,name,category,main_category,deadline,goal,launched,state,backers,country,usd_pledged,currency,pledged
0,1147015301,"""All We Had"" Gets Into Cannes -- $10 or More G...",Documentary,Film & Video,2009-05-20 21:50:00,300.0,2009-04-30 22:10:30,failed,4,US,40.00,USD,40.00
1,1100844465,daily digest,Documentary,Film & Video,2009-06-01 02:20:00,700.0,2009-05-04 21:14:28,successful,14,US,700.00,USD,700.00
2,1316334968,Drive A Faster Car 2.0,Nonfiction,Publishing,2009-06-04 06:00:00,1000.0,2009-05-04 22:51:31,successful,32,US,1367.00,USD,1367.00
3,1304906577,Accidental to Edinburgh - PHASE 1: AIRFARE,Theater,Theater,2009-06-05 05:59:00,6000.0,2009-04-30 22:22:43,successful,24,US,6575.00,USD,6575.00
4,1099226462,Logical Guess Pictures' 2nd Horror Movie!,Film & Video,Film & Video,2009-06-06 00:45:00,500.0,2009-04-30 01:32:55,successful,22,US,501.66,USD,501.66


####**1. How many rows of data are in the DataFrame?**

In [3]:
print('There are {} rows in the kickstarter dataset.'.format(kickstarter.shape[0]))

There are 65418 rows in the kickstarter dataset.


####**2. What are the names and data types of the columns?**

In [4]:
print('The column names are:\n', pd.Series(kickstarter.columns))

The column names are:
 0                id
1              name
2          category
3     main_category
4          deadline
5              goal
6          launched
7             state
8           backers
9           country
10      usd_pledged
11         currency
12          pledged
dtype: object


####**3. Do any of the columns contain null values?**

In [5]:
kickstarter.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65418 entries, 0 to 65417
Data columns (total 13 columns):
id               65418 non-null int64
name             65418 non-null object
category         65418 non-null object
main_category    65418 non-null object
deadline         65418 non-null object
goal             65418 non-null float64
launched         65418 non-null object
state            65418 non-null object
backers          65418 non-null int64
country          65418 non-null object
usd_pledged      64673 non-null float64
currency         65418 non-null object
pledged          65418 non-null float64
dtypes: float64(3), int64(2), object(8)
memory usage: 6.5+ MB


The only column with missing values is usd_pledged.

####**4. Find all successful documentary projects and sort them by the amount pledged. Print the top 10 highest pledges.**

In [6]:
successful_documentaries = (kickstarter['category'] == 'Documentary') & (kickstarter['state'] == 'successful')
top_docs = kickstarter.iloc[successful_documentaries.values, [1, 2, 5, 7, 8, 9, 10]].sort_values(by = 'usd_pledged', ascending = False)[:10]
top_docs.index = [str(i) for i in range(1, 11)]
top_docs

,name,category,goal,state,backers,country,usd_pledged
1,BRIDEGROOM - An American Love Story,Documentary,300000.0,successful,6508,US,384376.0
2,"""The Wrecking Crew"" The Untold Story of Rock &...",Documentary,250000.0,successful,4245,US,313158.0
3,The Killing of Tony Blair,Documentary,50000.0,successful,4459,GB,255150.0
4,The Culture High,Documentary,190000.0,successful,3448,US,240022.0
5,ILEANA'S SMILE,Documentary,133000.0,successful,1173,US,177421.0
6,Rewind To Fast-Forward,Documentary,137000.0,successful,4395,US,176746.0
7,Linsanity: the Movie,Documentary,117000.0,successful,2102,US,167917.0
8,El Voto Hispano | The Documentary,Documentary,158000.0,successful,461,US,165401.0
9,"PlantPure Nation, a Documentary with Dr. T. Co...",Documentary,150000.0,successful,1099,US,163930.0
10,BREXIT THE MOVIE,Documentary,100000.0,successful,1685,GB,162393.0


####**5. Create a new column named average_per_backer and set its value to the total amount pledged / number of backers. What happened to the rows with 0 backers? How can this be dealt with?**

In [16]:
kickstarter['average_per_backer'] = kickstarter['usd_pledged'] / kickstarter['backers']
kickstarter

,id,name,category,main_category,deadline,goal,launched,state,backers,country,usd_pledged,currency,pledged,average_per_backer
0,1147015301,"""All We Had"" Gets Into Cannes -- $10 or More G...",Documentary,Film & Video,2009-05-20 21:50:00,300.0,2009-04-30 22:10:30,failed,4,US,40.00,USD,40.00,10.000000
1,1100844465,daily digest,Documentary,Film & Video,2009-06-01 02:20:00,700.0,2009-05-04 21:14:28,successful,14,US,700.00,USD,700.00,50.000000
2,1316334968,Drive A Faster Car 2.0,Nonfiction,Publishing,2009-06-04 06:00:00,1000.0,2009-05-04 22:51:31,successful,32,US,1367.00,USD,1367.00,42.718750
3,1304906577,Accidental to Edinburgh - PHASE 1: AIRFARE,Theater,Theater,2009-06-05 05:59:00,6000.0,2009-04-30 22:22:43,successful,24,US,6575.00,USD,6575.00,273.958333
4,1099226462,Logical Guess Pictures' 2nd Horror Movie!,Film & Video,Film & Video,2009-06-06 00:45:00,500.0,2009-04-30 01:32:55,successful,22,US,501.66,USD,501.66,22.802727
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65413,1107492255,DODEKA - music finally within everyone's reach!,Sound,Technology,2017-02-03 14:51:00,55000.0,2016-12-05 14:51:00,live,9,CH,1022.12,CHF,9287.00,113.568889
65414,1373187022,World's Smallest Water-resistant Magnetic Came...,Gadgets,Technology,2017-02-03 17:23:11,2000.0,2016-12-05 17:23:11,canceled,148,US,5474.00,USD,5552.00,36.986486
65415,1212504494,Varnville Central Station - Guest House #1,Product Design,Design,2017-02-03 20:17:57,30000.0,2016-12-05 20:17:57,live,0,US,0.00,USD,0.00,NaN
65416,1347446277,Luminaria emergencia Led con mensajes.,Technology,Technology,2017-02-03 21:51:02,30000.0,2016-12-05 21:51:02,live,0,ES,0.00,EUR,0.00,NaN


The resulting column has introduced NaNs where backers = 0. We could fix this with an if/else statement.

In [19]:
# initialize, don't grow lists, arrays, or series
kickstarter['average_per_backer'] = 0
non_zero_backers = kickstarter['backers'] != 0
kickstarter['average_per_backer'][non_zero_backers] = kickstarter['usd_pledged'] / kickstarter['backers']
kickstarter

,id,name,category,main_category,deadline,goal,launched,state,backers,country,usd_pledged,currency,pledged,average_per_backer
0,1147015301,"""All We Had"" Gets Into Cannes -- $10 or More G...",Documentary,Film & Video,2009-05-20 21:50:00,300.0,2009-04-30 22:10:30,failed,4,US,40.00,USD,40.00,10.000000
1,1100844465,daily digest,Documentary,Film & Video,2009-06-01 02:20:00,700.0,2009-05-04 21:14:28,successful,14,US,700.00,USD,700.00,50.000000
2,1316334968,Drive A Faster Car 2.0,Nonfiction,Publishing,2009-06-04 06:00:00,1000.0,2009-05-04 22:51:31,successful,32,US,1367.00,USD,1367.00,42.718750
3,1304906577,Accidental to Edinburgh - PHASE 1: AIRFARE,Theater,Theater,2009-06-05 05:59:00,6000.0,2009-04-30 22:22:43,successful,24,US,6575.00,USD,6575.00,273.958333
4,1099226462,Logical Guess Pictures' 2nd Horror Movie!,Film & Video,Film & Video,2009-06-06 00:45:00,500.0,2009-04-30 01:32:55,successful,22,US,501.66,USD,501.66,22.802727
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65413,1107492255,DODEKA - music finally within everyone's reach!,Sound,Technology,2017-02-03 14:51:00,55000.0,2016-12-05 14:51:00,live,9,CH,1022.12,CHF,9287.00,113.568889
65414,1373187022,World's Smallest Water-resistant Magnetic Came...,Gadgets,Technology,2017-02-03 17:23:11,2000.0,2016-12-05 17:23:11,canceled,148,US,5474.00,USD,5552.00,36.986486
65415,1212504494,Varnville Central Station - Guest House #1,Product Design,Design,2017-02-03 20:17:57,30000.0,2016-12-05 20:17:57,live,0,US,0.00,USD,0.00,0.000000
65416,1347446277,Luminaria emergencia Led con mensajes.,Technology,Technology,2017-02-03 21:51:02,30000.0,2016-12-05 21:51:02,live,0,ES,0.00,EUR,0.00,0.000000



####**6. Drop all rows with 0 backers then repeat the previous exercise.**


In [20]:
# drop column from previous exercise
del kickstarter['average_per_backer']
kickstarter_drop_zero_backers = kickstarter[kickstarter['backers'] != 0]
kickstarter_drop_zero_backers.head()

,id,name,category,main_category,deadline,goal,launched,state,backers,country,usd_pledged,currency,pledged
0,1147015301,"""All We Had"" Gets Into Cannes -- $10 or More G...",Documentary,Film & Video,2009-05-20 21:50:00,300.0,2009-04-30 22:10:30,failed,4,US,40.00,USD,40.00
1,1100844465,daily digest,Documentary,Film & Video,2009-06-01 02:20:00,700.0,2009-05-04 21:14:28,successful,14,US,700.00,USD,700.00
2,1316334968,Drive A Faster Car 2.0,Nonfiction,Publishing,2009-06-04 06:00:00,1000.0,2009-05-04 22:51:31,successful,32,US,1367.00,USD,1367.00
3,1304906577,Accidental to Edinburgh - PHASE 1: AIRFARE,Theater,Theater,2009-06-05 05:59:00,6000.0,2009-04-30 22:22:43,successful,24,US,6575.00,USD,6575.00
4,1099226462,Logical Guess Pictures' 2nd Horror Movie!,Film & Video,Film & Video,2009-06-06 00:45:00,500.0,2009-04-30 01:32:55,successful,22,US,501.66,USD,501.66


In [21]:
kickstarter_drop_zero_backers['average_per_backer'] = kickstarter['usd_pledged'] / kickstarter['backers']
kickstarter_drop_zero_backers.head()

,id,name,category,main_category,deadline,goal,launched,state,backers,country,usd_pledged,currency,pledged,average_per_backer
0,1147015301,"""All We Had"" Gets Into Cannes -- $10 or More G...",Documentary,Film & Video,2009-05-20 21:50:00,300.0,2009-04-30 22:10:30,failed,4,US,40.00,USD,40.00,10.000000
1,1100844465,daily digest,Documentary,Film & Video,2009-06-01 02:20:00,700.0,2009-05-04 21:14:28,successful,14,US,700.00,USD,700.00,50.000000
2,1316334968,Drive A Faster Car 2.0,Nonfiction,Publishing,2009-06-04 06:00:00,1000.0,2009-05-04 22:51:31,successful,32,US,1367.00,USD,1367.00,42.718750
3,1304906577,Accidental to Edinburgh - PHASE 1: AIRFARE,Theater,Theater,2009-06-05 05:59:00,6000.0,2009-04-30 22:22:43,successful,24,US,6575.00,USD,6575.00,273.958333
4,1099226462,Logical Guess Pictures' 2nd Horror Movie!,Film & Video,Film & Video,2009-06-06 00:45:00,500.0,2009-04-30 01:32:55,successful,22,US,501.66,USD,501.66,22.802727


####**7. Create a crosstab to get a count of records for each combination of state and category.**

In [11]:
pd.crosstab(kickstarter['category'], kickstarter['state'])

state,canceled,failed,live,successful,suspended,undefined
category,,,,,,
3D Printing,17,59,3,43,2,1
Academic,21,86,3,43,4,0
Accessories,48,238,16,121,2,1
Action,14,79,1,16,0,0
Animals,2,31,0,11,1,0
...,...,...,...,...,...,...
Woodworking,10,125,4,38,2,0
Workshops,0,12,1,8,1,0
World Music,31,174,2,168,1,0


## nycflights13
This data consists of information about all flights that departed NYC in 2013. Download two datasets: [flights](https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/flights.csv) and [airlines](https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/airlines.csv).

1. Merge these two DataFrames into a single DataFrame: nycflights13.
2. Are there any missing values in this DataFrame?
If so, drop them.

In [12]:
flights_url = 'https://raw.githubusercontent.com/kristi-thinkful/thinkful-solutions/master/python_III_pandas/data/flights.csv'
airlines_url = 'https://raw.githubusercontent.com/kristi-thinkful/thinkful-solutions/master/python_III_pandas/data/airlines.csv'
flights = pd.read_csv(flights_url)
airlines = pd.read_csv(airlines_url)
flights.head()
airlines.head()

,year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,flight,tailnum,origin,dest,air_time,distance,hour,minute,time_hour
0,2013,1,1,517.0,515,2.0,830.0,819,11.0,UA,1545,N14228,EWR,IAH,227.0,1400,5,15,2013-01-01 05:00:00
1,2013,1,1,533.0,529,4.0,850.0,830,20.0,UA,1714,N24211,LGA,IAH,227.0,1416,5,29,2013-01-01 05:00:00
2,2013,1,1,542.0,540,2.0,923.0,850,33.0,AA,1141,N619AA,JFK,MIA,160.0,1089,5,40,2013-01-01 05:00:00
3,2013,1,1,544.0,545,-1.0,1004.0,1022,-18.0,B6,725,N804JB,JFK,BQN,183.0,1576,5,45,2013-01-01 05:00:00
4,2013,1,1,554.0,600,-6.0,812.0,837,-25.0,DL,461,N668DN,LGA,ATL,116.0,762,6,0,2013-01-01 06:00:00


,carrier,name
0,9E,Endeavor Air Inc.
1,AA,American Airlines Inc.
2,AS,Alaska Airlines Inc.
3,B6,JetBlue Airways
4,DL,Delta Air Lines Inc.


####**1. Merge these two DataFrames into a single DataFrame: nycflights13.** 

In [22]:
nycflights13 = pd.merge(airlines, flights)
nycflights13.head()

,carrier,name,year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,flight,tailnum,origin,dest,air_time,distance,hour,minute,time_hour
0,9E,Endeavor Air Inc.,2013,1,1,810.0,810,0.0,1048.0,1037,11.0,3538,N915XJ,JFK,MSP,189.0,1029,8,10,2013-01-01 08:00:00
1,9E,Endeavor Air Inc.,2013,1,1,1451.0,1500,-9.0,1634.0,1636,-2.0,4105,N8444F,JFK,IAD,57.0,228,15,0,2013-01-01 15:00:00
2,9E,Endeavor Air Inc.,2013,1,1,1452.0,1455,-3.0,1637.0,1639,-2.0,3295,N920XJ,JFK,BUF,68.0,301,14,55,2013-01-01 14:00:00
3,9E,Endeavor Air Inc.,2013,1,1,1454.0,1500,-6.0,1635.0,1636,-1.0,3843,N8409N,JFK,SYR,57.0,209,15,0,2013-01-01 15:00:00
4,9E,Endeavor Air Inc.,2013,1,1,1507.0,1515,-8.0,1651.0,1656,-5.0,3792,N8631E,JFK,ROC,66.0,264,15,15,2013-01-01 15:00:00


####**2. Are there any missing values in this DataFrame? If so, drop them.**

In [23]:
nycflights13.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 336776 entries, 0 to 336775
Data columns (total 20 columns):
carrier           336776 non-null object
name              336776 non-null object
year              336776 non-null int64
month             336776 non-null int64
day               336776 non-null int64
dep_time          328521 non-null float64
sched_dep_time    336776 non-null int64
dep_delay         328521 non-null float64
arr_time          328063 non-null float64
sched_arr_time    336776 non-null int64
arr_delay         327346 non-null float64
flight            336776 non-null int64
tailnum           334264 non-null object
origin            336776 non-null object
dest              336776 non-null object
air_time          327346 non-null float64
distance          336776 non-null int64
hour              336776 non-null int64
minute            336776 non-null int64
time_hour         336776 non-null object
dtypes: float64(5), int64(9), object(6)
memory usage: 54.0+ MB


There are missing values in columns dep_time, dep_delay, arr_time, arr_delay, tailnum, and air_time.

In [28]:
nycflights13.dropna(inplace = True)
nycflights13.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 327346 entries, 0 to 336775
Data columns (total 20 columns):
carrier           327346 non-null object
name              327346 non-null object
year              327346 non-null int64
month             327346 non-null int64
day               327346 non-null int64
dep_time          327346 non-null float64
sched_dep_time    327346 non-null int64
dep_delay         327346 non-null float64
arr_time          327346 non-null float64
sched_arr_time    327346 non-null int64
arr_delay         327346 non-null float64
flight            327346 non-null int64
tailnum           327346 non-null object
origin            327346 non-null object
dest              327346 non-null object
air_time          327346 non-null float64
distance          327346 non-null int64
hour              327346 non-null int64
minute            327346 non-null int64
time_hour         327346 non-null object
dtypes: float64(5), int64(9), object(6)
memory usage: 52.4+ MB


## census
For this portion, download two datasets: [census_population](https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/state-populations.csv) and [census_regions](https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/census-divisions.csv).

1. Merge these two DataFrames into a single DataFrame called census.
2. Re-shape census so that one column contains all population measures, and another the year attributes.
3. Group the data by year and summarize it.
4. Group the data by region, division and year and summarize it.

In [29]:
census_pop_url = 'https://raw.githubusercontent.com/kristi-thinkful/thinkful-solutions/master/python_III_pandas/data/state-populations.csv'
census_reg_url = 'https://raw.githubusercontent.com/kristi-thinkful/thinkful-solutions/master/python_III_pandas/data/census-divisions.csv'
census_pop = pd.read_csv(census_pop_url)
census_reg = pd.read_csv(census_reg_url)
census_pop.head()
census_reg.head()

,state,2010,2011,2012,2013,2014,2015,2016
0,Alabama,4785492,4799918,4815960,4829479,4843214,4853875,4863300
1,Alaska,714031,722713,731089,736879,736705,737709,741894
2,Arizona,6408312,6467163,6549634,6624617,6719993,6817565,6931071
3,Arkansas,2921995,2939493,2950685,2958663,2966912,2977853,2988248
4,California,37332685,37676861,38011074,38335203,38680810,38993940,39250017


,state,region,division
0,Connecticut,Northeast,New England
1,Maine,Northeast,New England
2,Massachusetts,Northeast,New England
3,New Hampshire,Northeast,New England
4,Rhode Island,Northeast,New England


####**1. Merge these two DataFrames into a single DataFrame called census.**

In [30]:
census = pd.merge(census_reg, census_pop)
census.head()

,state,region,division,2010,2011,2012,2013,2014,2015,2016
0,Connecticut,Northeast,New England,3579899,3589893,3593795,3596003,3591873,3584730,3576452
1,Maine,Northeast,New England,1327730,1328231,1328895,1329076,1330719,1329453,1331479
2,Massachusetts,Northeast,New England,6565524,6611923,6658008,6706786,6749911,6784240,6811779
3,New Hampshire,Northeast,New England,1316872,1318473,1321182,1322687,1328743,1330111,1334795
4,Rhode Island,Northeast,New England,1053337,1052451,1052901,1053033,1054480,1055607,1056426


####**2. Re-shape census so that one column contains all population measures, and another the year attributes.**


In [32]:
census_long = pd.melt(frame = census, id_vars = ['state', 'region', 'division'], var_name = 'year', value_name = 'population')
census_long.head()

,state,region,division,year,population
0,Connecticut,Northeast,New England,2010,3579899
1,Maine,Northeast,New England,2010,1327730
2,Massachusetts,Northeast,New England,2010,6565524
3,New Hampshire,Northeast,New England,2010,1316872
4,Rhode Island,Northeast,New England,2010,1053337


####**3. Group the data by year and summarize it.**

In [33]:
census_long.groupby(by = ['year']).describe()

population                              ...                                  
          count          mean           std  ...        50%        75%         max
year                                         ...                                  
2010       51.0  6.065651e+06  6.839536e+06  ...  4348662.0  6654375.0  37332685.0
2011       51.0  6.111046e+06  6.907730e+06  ...  4369354.0  6717221.5  37676861.0
2012       51.0  6.156831e+06  6.974942e+06  ...  4384799.0  6776617.0  38011074.0
2013       51.0  6.200096e+06  7.038774e+06  ...  4400477.0  6837396.0  38335203.0
2014       51.0  6.246342e+06  7.110811e+06  ...  4413057.0  6902053.5  38680810.0
2015       51.0  6.292091e+06  7.181603e+06  ...  4424611.0  6988927.5  38993940.0
2016       51.0  6.335834e+06  7.243849e+06  ...  4436974.0  7109535.5  39250017.0

[7 rows x 8 columns]

####**4. Group the data by region, division and year and summarize it.**

In [34]:
census_long.groupby(by = ['region']).describe()

population                ...                       
               count          mean  ...        75%         max
region                              ...                       
Midwest         84.0  5.625171e+06  ...  7443843.0  12879505.0
Northeast       63.0  6.211900e+06  ...  8930211.0  19747183.0
South          119.0  6.972298e+06  ...  8342479.5  27862596.0
West            91.0  5.716867e+06  ...  5399233.5  39250017.0

[4 rows x 8 columns]